In [57]:
import findspark
findspark.init()

In [58]:
import pyspark
from pyspark.sql import *

from pyspark.ml.feature import StopWordsRemover
from pyspark import SparkConf, SparkContext
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from itertools import islice
from pyspark.mllib.feature import Word2Vec

from pyspark.sql import SparkSession

from pyspark.sql import SQLContext, Row
from pyspark.ml.feature import CountVectorizer
from pyspark.mllib.clustering import LDA, LDAModel
#from pyspark.mllib.linalg import Vector, Vectors
#from pyspark.ml.linalg import Vector, Vectors
from pyspark.ml.linalg import Vector, Vectors, VectorUDT

In [59]:
import numpy as np

In [60]:
sc = pyspark.SparkContext.getOrCreate()

In [61]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [62]:
import random
num_samples = 1000

def inside(p):     
    x, y = random.random(), random.random()
    return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

3.104


In [63]:
TOTAL = 100
dots = sc.parallelize([2.0 * np.random.random(2) - 1.0 for i in range(TOTAL)]).cache()
print("Number of random points:", dots.count())

stats = dots.stats()

print('Mean:', stats.mean())
print('stdev:', stats.stdev())

Number of random points: 100
Mean: [ 0.02130042 -0.00543566]
stdev: [0.55464035 0.55158877]


In [64]:
sc.stop()

In [65]:
num_rows_to_show = 20
text_file = 'data/listings.csv'

In [66]:
sc = SparkContext() # pyspark.SparkContext.getOrCreate()
spark = SparkSession(sc) # SparkSession.builder.appName('ops').getOrCreate()

In [67]:
# path = "sample_text_LDA.txt"

# data = sc.textFile(path).zipWithIndex().map(lambda (words,idd): Row(idd= idd, words = words.split(" ")))
# docDF = spark.createDataFrame(data)

In [68]:
df = spark.read.csv(text_file, inferSchema=True, header=True)
corpusMy = df.select("id", "name").dropna(subset="name")

In [69]:
# importing some libraries
import os
import pandas as pd
import pyspark
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# stuff we'll need for text processing

from nltk.corpus import stopwords
import re as re
#from pyspark.ml.feature import CountVectorizer , IDF

# stuff we'll need for building the model

#from pyspark.mllib.linalg import Vector, Vectors
#from pyspark.mllib.clustering import LDA, LDAModel

### Approach 1

https://medium.com/@connectwithghosh/topic-modelling-with-latent-dirichlet-allocation-lda-in-pyspark-2cb3ebd5678e

In [24]:
data = sqlContext.read.format("csv") \
   .options(header='true', inferschema='true') \
   .load(os.path.realpath(text_file))

In [25]:
reviews = data.rdd.map(lambda x : x['name']).filter(lambda x: x is not None)

In [26]:
StopWords = stopwords.words("english")

tokens = reviews                                                   \
    .map( lambda document: document.strip().lower())               \
    .map( lambda document: re.split(" ", document))          \
    .map( lambda word: [x for x in word if x.isalpha()])           \
    .map( lambda word: [x for x in word if len(x) > 3] )           \
    .map( lambda word: [x for x in word if x not in StopWords])    \
    .zipWithIndex()

In [27]:
df_txts = sqlContext.createDataFrame(tokens, ["list_of_words",'index'])

# TF
cv = CountVectorizer(inputCol="list_of_words", outputCol="raw_features", vocabSize=5000, minDF=10.0)
cvmodel = cv.fit(df_txts)

result_cv = cvmodel.transform(df_txts)

# IDF

idf = IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(result_cv)
result_tfidf = idfModel.transform(result_cv) 

#### PROBLEM IS HERE

In [28]:
num_topics = 10
max_iterations = 100

lda_model = LDA.train(result_tfidf[['index','features']].rdd.map(list), k=num_topics, maxIterations=max_iterations)

Py4JJavaError: An error occurred while calling o370.trainLDAModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 18.0 failed 1 times, most recent failure: Lost task 0.0 in stage 18.0 (TID 18, localhost, executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for pyspark.ml.linalg.SparseVector)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:707)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:175)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.mllib.api.python.SerDeBase$$anonfun$pythonToJava$1$$anonfun$apply$2.apply(PythonMLLibAPI.scala:1350)
	at org.apache.spark.mllib.api.python.SerDeBase$$anonfun$pythonToJava$1$$anonfun$apply$2.apply(PythonMLLibAPI.scala:1349)
	at scala.collection.I

### Approache 2

In [29]:
df = spark.read.csv(text_file, inferSchema=True, header=True)
corpus2= df.select("id", "name").dropna(subset="name")

In [30]:
tokenizer = Tokenizer(inputCol="name", outputCol="words")
docDF = tokenizer.transform(corpus2)

In [31]:
Vector = CountVectorizer(inputCol="words", outputCol="vectors")
model = Vector.fit(docDF)
result = model.transform(docDF)

In [32]:
result.show(10, False)

+-----+-------------------------------------------------+---------------------------------------------------------+--------------------------------------------------------------------+
|id   |name                                             |words                                                    |vectors                                                             |
+-----+-------------------------------------------------+---------------------------------------------------------+--------------------------------------------------------------------+
|2818 |Quiet Garden View Room & Super Fast WiFi         |[quiet, garden, view, room, &, super, fast, wifi]        |(7254,[8,9,17,31,51,141,236,1186],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|
|20168|100%Centre-Studio 1 Private Floor/Bathroom       |[100%centre-studio, 1, private, floor/bathroom]          |(7254,[20,103,1930,2448],[1.0,1.0,1.0,1.0])                         |
|25428|Lovely apt in City Centre (Jordaan)              |[lovely, apt, in, 

In [33]:
tokenizer = Tokenizer(inputCol="name", outputCol="raw_words")

In [34]:
wordsData = tokenizer.transform(result) #corpus

In [35]:
locale = spark._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))

StopWordsRemover.loadDefaultStopWords("english")

remover = StopWordsRemover(inputCol="raw_words", outputCol="words2")
wordsData = remover.transform(wordsData)

In [36]:
hashingTF = HashingTF(inputCol="words2", outputCol="rawFeatures", numFeatures=10000)
featurizedData = hashingTF.transform(wordsData)

In [37]:
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=1)
idfModel = idf.fit(featurizedData)

tfidf = idfModel.transform(featurizedData)

In [38]:
tfidf.show(10, False)

+-----+-------------------------------------------------+---------------------------------------------------------+--------------------------------------------------------------------+---------------------------------------------------------+-----------------------------------------------------+----------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id   |name                                             |words                                                    |vectors                                                             |raw_words                                                |words2                                               |rawFeatures                                                                       |features                                    

#### PROBLEM IS HERE

In [40]:
num_topics = 10
max_iterations = 100

lda_model = LDA.train(tfidf[['id','features']].rdd, k=num_topics, maxIterations=max_iterations)

Py4JJavaError: An error occurred while calling o607.trainLDAModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 28.0 failed 1 times, most recent failure: Lost task 0.0 in stage 28.0 (TID 27, localhost, executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for pyspark.ml.linalg.SparseVector)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:707)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:175)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.mllib.api.python.SerDeBase$$anonfun$pythonToJava$1$$anonfun$apply$2.apply(PythonMLLibAPI.scala:1350)
	at org.apache.spark.mllib.api.python.SerDeBase$$anonfun$pythonToJava$1$$anonfun$apply$2.apply(PythonMLLibAPI.scala:1349)
	at scala.collection.I

#### AND HERE

In [47]:
# data = sc.textFile(text_file)
# parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))

df = spark.read.csv(text_file, inferSchema=True, header=True)
corpusMy = df.select("id", "name").dropna(subset="name")

In [56]:
corpusMy.select("id", "name").show(10)
test = corpusMy.select("id", "name").rdd.map(lambda x: [x[1], x[0]]).collect().parralelize()
print(test)


+-----+--------------------+
|   id|                name|
+-----+--------------------+
| 2818|Quiet Garden View...|
|20168|100%Centre-Studio...|
|25428|Lovely apt in Cit...|
|27886|Romantic, stylish...|
|28658|Cosy guest room n...|
|28871|Comfortable doubl...|
|29051|Comfortable singl...|
|31080|2-story apartment...|
|38266|Nice and quiet pl...|
|41125|Amsterdam Center ...|
+-----+--------------------+
only showing top 10 rows



AttributeError: 'list' object has no attribute 'parralelize'

In [54]:
lda_model = LDA.train(test, k=num_topics, maxIterations=max_iterations)

Py4JJavaError: An error occurred while calling o1048.trainLDAModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 45.0 failed 1 times, most recent failure: Lost task 0.0 in stage 45.0 (TID 44, localhost, executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for pyspark.sql.types._create_row)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:707)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:175)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.mllib.api.python.SerDeBase$$anonfun$pythonToJava$1$$anonfun$apply$2.apply(PythonMLLibAPI.scala:1350)
	at org.apache.spark.mllib.api.python.SerDeBase$$anonfun$pythonToJava$1$$anonfun$apply$2.apply(PythonMLLibAPI.scala:1349)
	at scala.collection.I

In [43]:
#corpus = result_tfidf.select("index", "features").rdd.map(lambda x,y: [x,Vectors.fromML(y)]).cache()
corpus = result_tfidf.select("index", "features").rdd.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()
result_tfidf.select("index", "features").head()

num_topics = 10
max_iterations = 100

lda_model = LDA.train(corpus, k=num_topics, maxIterations=max_iterations)

Py4JJavaError: An error occurred while calling o757.trainLDAModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 31.0 failed 1 times, most recent failure: Lost task 0.0 in stage 31.0 (TID 30, localhost, executor driver): net.razorvine.pickle.PickleException: expected zero arguments for construction of ClassDict (for pyspark.ml.linalg.SparseVector)
	at net.razorvine.pickle.objects.ClassDictConstructor.construct(ClassDictConstructor.java:23)
	at net.razorvine.pickle.Unpickler.load_reduce(Unpickler.java:707)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:175)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:99)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:112)
	at org.apache.spark.mllib.api.python.SerDeBase$$anonfun$pythonToJava$1$$anonfun$apply$2.apply(PythonMLLibAPI.scala:1350)
	at org.apache.spark.mllib.api.python.SerDeBase$$anonfun$pythonToJava$1$$anonfun$apply$2.apply(PythonMLLibAPI.scala:1349)
	at scala.collection.I

In [36]:
# topics = ldaModel.topicsMatrix()
# vocabArray = model.vocabulary

In [ ]:
# wordNumbers = 10  # number of words per topic
# topicIndices = sc.parallelize(ldaModel.describeTopics(maxTermsPerTopic = wordNumbers))

# def topic_render(topic):  # specify vector id of words to actual words
#     terms = topic[0]
#     result = []
#     for i in range(wordNumbers):
#         term = vocabArray[terms[i]]
#         result.append(term)
#     return result

# topics_final = topicIndices.map(lambda topic: topic_render(topic)).collect()

# for topic in range(len(topics_final)):
#     print ("Topic" + str(topic) + ":")
#     for term in topics_final[topic]:
#         print (term)
#     print ('\n')

### Approache 3
https://stackoverflow.com/questions/42051184/latent-dirichlet-allocation-lda-in-spark

In [91]:
from pyspark.mllib.clustering import LDA, LDAModel
from pyspark.mllib.linalg import Vectors

# Load and parse the data
data = sc.textFile("./sample_lda_data.txt")
parsedData = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))
# Index documents with unique IDs
corpus = parsedData.zipWithIndex().map(lambda x: [x[1], x[0]]).cache()



In [89]:
a = data.map(lambda line: Vectors.dense([float(x) for x in line.strip().split(' ')]))

In [92]:
corpus.toDF().show(10)

+---+--------------------+
| _1|                  _2|
+---+--------------------+
|  0|[1.0,2.0,6.0,0.0,...|
|  1|[1.0,3.0,0.0,1.0,...|
|  2|[1.0,4.0,1.0,0.0,...|
|  3|[2.0,1.0,0.0,3.0,...|
|  4|[3.0,1.0,1.0,9.0,...|
|  5|[4.0,2.0,0.0,3.0,...|
|  6|[2.0,1.0,0.0,3.0,...|
|  7|[1.0,1.0,1.0,9.0,...|
|  8|[4.0,4.0,0.0,3.0,...|
|  9|[2.0,8.0,2.0,0.0,...|
+---+--------------------+
only showing top 10 rows



In [93]:
corpus.toDF().show(10)

+---+--------------------+
| _1|                  _2|
+---+--------------------+
|  0|[1.0,2.0,6.0,0.0,...|
|  1|[1.0,3.0,0.0,1.0,...|
|  2|[1.0,4.0,1.0,0.0,...|
|  3|[2.0,1.0,0.0,3.0,...|
|  4|[3.0,1.0,1.0,9.0,...|
|  5|[4.0,2.0,0.0,3.0,...|
|  6|[2.0,1.0,0.0,3.0,...|
|  7|[1.0,1.0,1.0,9.0,...|
|  8|[4.0,4.0,0.0,3.0,...|
|  9|[2.0,8.0,2.0,0.0,...|
+---+--------------------+
only showing top 10 rows



In [94]:
corpusMy.select("id", "name").show(10)
test = corpusMy.select("id", "name").rdd.map(lambda x: [x[1], x[0]]).collect().parralelize()
print(test)


+-----+--------------------+
|   id|                name|
+-----+--------------------+
| 2818|Quiet Garden View...|
|20168|100%Centre-Studio...|
|25428|Lovely apt in Cit...|
|27886|Romantic, stylish...|
|28658|Cosy guest room n...|
|28871|Comfortable doubl...|
|29051|Comfortable singl...|
|31080|2-story apartment...|
|38266|Nice and quiet pl...|
|41125|Amsterdam Center ...|
+-----+--------------------+
only showing top 10 rows



AttributeError: 'list' object has no attribute 'parralelize'

In [95]:
# Cluster the documents into three topics using LDA
ldaModel = LDA.train(corpus, k=3)


In [96]:

# Output topics. Each is a distribution over words (matching word count vectors)
print("Learned topics (as distributions over vocab of " + str(ldaModel.vocabSize())
      + " words):")
topics = ldaModel.topicsMatrix()
for topic in range(3):
    print("Topic " + str(topic) + ":")
    for word in range(0, ldaModel.vocabSize()):
        print(" " + str(topics[word][topic]))

Learned topics (as distributions over vocab of 11 words):
Topic 0:
 12.766212964867128
 5.404062581296598
 1.2080547519291418
 11.224732916708373
 7.271996259247922
 6.272972206891282
 13.592066657244391
 4.3040630266363955
 3.5758103017733465
 6.387035011756948
 14.688940666297647
Topic 1:
 5.447136988801134
 10.34828688951476
 2.4032038384806174
 19.72500381914468
 7.548150077199549
 5.91222146284111
 9.780236217644898
 1.4578817678328848
 2.677072435605984
 11.479444559843477
 10.156443464835043
Topic 2:
 7.786650046331737
 13.24765052918864
 8.388741409590242
 9.050263264146945
 10.179853663552528
 9.814806330267606
 7.627697125110712
 4.238055205530719
 1.7471172626206697
 6.133520428399574
 8.15461586886731


In [58]:
tokenizer = Tokenizer(inputCol="name", outputCol="words")
docDF = tokenizer.transform(corpus2)
Vector = CountVectorizer(inputCol="words", outputCol="vectors")
model = Vector.fit(docDF)
result = model.transform(docDF)

In [61]:
corpus = result.select("id", "vectors").rdd.map(lambda x,y: [x,Vectors.fromML(y)]).cache()

#### PROBLEM IS HERE

In [62]:
ldaModel = LDA.train(corpus, k=3,maxIterations=100,optimizer='online')

Py4JJavaError: An error occurred while calling o1392.trainLDAModel.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 35.0 failed 1 times, most recent failure: Lost task 0.0 in stage 35.0 (TID 33, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 393, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/spark/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwargs)
TypeError: <lambda>() missing 1 required positional argument: 'y'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.